# Entity-<issue #>: 
## LEAR: <registration number, eg CP0000001>
## - task description

**Note:** The historical change/correction filings are not in production yet, so this will need to be done via direct changes to the services DB.

## setup the environment

In [ ]:
%run ./bcr-business-setup.ipynb

## set the registration number of the business eg. CP0001234
vv-- set the registration number here

In [ ]:
registration_number = 'CP0001790'

## lookup the business and ensure it is the one you want to change

In [ ]:
business = legal_api.models.Business.find_by_identifier(registration_number)
business.id, business.identifier, business.legal_name

## lookup the correction filing
the filing contents will be echo'ed below for reference

In [ ]:
filing = get_latest_correction_filing(business.id)
filing.id, filing.status, filing.paper_only, filing.transaction_id, filing.filing_json

### the following checks if the filing has already been processed.
**if it has been processed already, then either of the following 2 calls will fail and processing will stop _IFF_ this is being run by papermill**

if this is being run from a job environment, it'll stop if the following checks are valid

In [ ]:
stop_on_true(filing.status=='COMPLETED', "Can't process a completed filing.")
# stop_on_true(filing.transaction_id is not None, "Can't process a filing already associated with a transaction.")
stop_on_false(filing.status=='PENDING_CORRECTION', "Can't process a filing that is not is the PENDING_CORRECTION state.")

## Model - for reference
run this cell if you want to see an image of the domain model
![model](img/legal.png)

In [ ]:
import pycountry
from legal_api import db
from legal_api.models import Address, Business, Filing, Office, Party, PartyRole
from sqlalchemy_continuum import versioning_manager


In [ ]:
uow = versioning_manager.unit_of_work(db.session)
transaction = uow.create_transaction(db.session)

## Change core busines details
uncomment and set any of the values

In [ ]:
# business.last_agm_date =
# business.last_ar_date = 
# business.legal_name = 
# business.legal_type =
# business.founding_date 
# business.dissolution_date =
# business.identifier = 
# business.tax_id = 
# business.fiscal_year_end_date = 

# Fix addresses

### find all of the addresses

Office types are in <span style="color: #0000ff">BLUE</span> ;
Address proporties are in <span style="color: #ff0000">RED</span> 

In [ ]:
offices = business.offices.all()
for office in offices:
    print(f"\x1b[34m{office.office_type}\x1b[0m")
    addresses = office.addresses.all()
    for address in addresses:
        print (f"\x1b[31mAddress.id=\x1b[0m{address.id}, \x1b[31mtype=\x1b[0m{address.address_type},\n\x1b[31mjson=\x1b[0m{address.json}")

### find the address to fix

In [ ]:
office = business.offices.filter_by(office_type='registeredOffice').one_or_none()
address_to_fix = office.addresses.filter_by(id=2007).one_or_none()
address_to_fix.json

### change the address info

In [ ]:
# address_to_fix.street = 'streetAddress'
# address_to_fix.street_additional = 'streetAddressAdditional'
# address_to_fix.city = 'addressCity'
# address_to_fix.region = 'addressRegion'
# address_to_fix.country = pycountry.countries.search_fuzzy('CA')[0].alpha_2
# address_to_fix.postal_code = 'postalCode'
# address_to_fix.delivery_instructions = 'deliveryInstructions'

# Directors

## fix exisiting director

In [ ]:
for director in legal_api.models.PartyRole.get_parties_by_role(business.id, 'director'):
    print (f"\x1b[31mDirector.id=\x1b[0m{director.id}, \x1b[31mName=\x1b[0m{director.party.first_name} {director.party.last_name}\n\x1b[31mjson=\x1b[0m{director.json}")

In [ ]:
director_to_fix = business.party_roles.filter_by(id=1520).one_or_none()
director_delivery_address = director_to_fix.party.delivery_address
director_mailing_address = director_to_fix.party.mailing_address
director_to_fix.json

In [ ]:
# director_to_fix.party.first_name = 
# director_to_fix.party.middle_initial = 
# director_to_fix.party.last_name = 
# director_to_fix.party.title = 
# director_to_fix.appointment_date = 
# director_to_fix.cessation_date =
# director_delivery_address = 

In [ ]:
# director_delivery_address.address_type = 'delivery'
# director_delivery_address.street = 
# director_delivery_address.street_additional = 
# director_delivery_address.city = 
# director_delivery_address.region = 
# director_delivery_address.country = pycountry.countries.search_fuzzy('CA')[0].alpha_2
# director_delivery_address.postal_code = 
# director_delivery_address.delivery_instructions = 

In [ ]:
# director_mailing_address.address_type = 'delivery'
# director_mailing_address.street = 
# director_mailing_address.street_additional = 
# director_mailing_address.city = 
# director_mailing_address.region = 
# director_mailing_address.country = pycountry.countries.search_fuzzy('CA')[0].alpha_2
# director_mailing_address.postal_code = 
# director_mailing_address.delivery_instructions = 

## add new director

In [ ]:
# new_party = Party(
#     first_name='firstname'.upper(),
#     middle_initial=None,
#     last_name='lastname'.upper(),
#     title=None
# )
# new_party_role = PartyRole(
#     role=PartyRole.RoleTypes.DIRECTOR.value,
#     appointment_date=datetime.date.fromisoformat('2018-12-31'),
#     cessation_date=None,
#     party=new_party
# )
# business.party_roles.append(new_party_role)

In [ ]:
# delivery_address = Address(
#     street='street'.upper(),
#     street_additional=None,
#     city='city'.upper(),
#     region='BC',
#     country=pycountry.countries.search_fuzzy('CA')[0].alpha_2,
#     postal_code='postal',
#     delivery_instructions=None,
#     address_type=Address.DELIVERY
# )

In [ ]:
# mailing_address = Address(
#     street='street'.upper(),
#     street_additional=None,
#     city='city'.upper(),
#     region='BC',
#     country=pycountry.countries.search_fuzzy('CA')[0].alpha_2,
#     postal_code='postal',
#     delivery_instructions=None,
#     address_type=Address.MAILING
# )

In [ ]:
# new_party_role.party.delivery_address = delivery_address
# new_party_role.party.mailing_address = mailing_address

# Save the changes to the database

In [ ]:
filing.transaction_id = transaction.id
filing.set_processed()

db.session.add(business)
db.session.add(filing)
db.session.commit()

### save a copy of this book as markdown

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
md_name = nb_name[:-6]+'.md'

In [ ]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2
